# 私有變數

In [ ]:
# 變數名稱開頭若有兩個底線，則為class的私有變數，系統不允許存取，系統會報錯
# 變數名稱開頭若有一個底線，則為默契上的私有變數，雖然系統允許存取，一般而言不建議存取
class Mine:
    def __init__(self, v1, v2, v3):
        self.x = v1
        self._y = v2 # 默契的私有變數
        self.__z = v3 # class的私有變數

    def show_variables(self):
        print('x:', self.x, '_y:', self._y, '__z:', self.__z)    

m = Mine(1, 2, 3)


print("m.x = ", m.x)   # 一般物件變數可以直接存取
print("m._y = ", m._y) # 默契的私有變數, 雖然不會報錯，但不要直接存取是共同的默契，所以不建議這麼作
# print("m.__z = ", m.__z) # # class的私有變數, 直接存會報錯

# 雖然透過下列這種方式可以存取class的私有變數，但強烈不建議這麼作
print("m._Mine__z = ", m._Mine__z) # class的私有變數, Python會透過名稱重整(Name Mangling)來防止外部直接訪問。
                                   # 事實上，Python 會將 __z 轉換成 _Mine__z

# 因此建議在 class 內, 寫一個 show variable的 method 來讀取默契的私有變數和class的私有變數
m.show_variables()

## Python 的變數命名規範
Naming Convention | Purpose
------------------|--------
var|General variable
_var|Single leading underscore: Indicates internal use, but not enforced <br>(Python suggests you avoid using it externally).
__var|Double leading underscore: Triggers name mangling, preventing accidental access from outside the class.
\_\_var\_\_|Double leading and trailing underscores.<br> Python reserved special variables or magic methods (e.g., \_\_name\_\_, \_\_init\_\_). Avoid using this for general variables.
var_|Trailing underscore: Used to avoid conflicts with Python keywords (e.g., class_).


# 使用 @property 修飾器來實作更靈活的變數
- A property in Python is a tool for creating managed attributes in classes.
- The @property decorator allows you to define getter, setter, and deleter methods for attributes.

In [ ]:
class Car:
    models = ['Toyota', 'Nissan', 'Honda']
    def __init__(self, model, color, mileage):
        if model in Car.models: # check model if it is in models
            self.model = model
        else:
            raise ValueError(f'{model} is not in {Car.models}')     
        self.color = color
        self.mileage = mileage    

    def __str__(self):
        return f'{self.model}, {self.color}, {self.mileage} km'
    

my_car = Car('Toyota', 'Red', 1000)
print(my_car)
# his_car = Car('Tesla', 'Red', 1000)
my_car.mileage = 500 # 違法調低里程數，需阻擋此更新，但此版本的 class 做不到
print(my_car)

In [ ]:
class Car:
    def __init__(self, model, color, mileage):
        self.model = model     
        self.color = color
        self.__mileage = mileage # 把 mileage 定義為私有變數, 外界不得讀取和更改
    def __str__(self):
        return f'{self.model}, {self.color}, {self.__mileage} km'

my_car = Car('Toyota', 'Red', 1000)
# print(my_car.__mileage) # Get error, __mileage 是一個class的私有變數
my_car.__mileage = 0 
print(my_car) # 還是沒改到

# 系統不是改變class的私有變數(因為它已經被改名成_Car__mileage)，而是新增一個物件變數 __mileage
print("__mileage:", my_car.__mileage)
print("_Car__mileage:", my_car._Car__mileage)
# 總之，不要嘗試直接存跟取私有變數，這是共同的默契

In [ ]:
class Car:
    def __init__(self, model, color, mileage):
        self.__model = model     
        self.color = color
        self.__mileage = mileage 
    def __str__(self):
        return f'{self.__model}, {self.color}, {self.__mileage} km'
    
    # ALT1 to get model value via method
    def get_model(self):
        return self.__model
    
    # ALT2 to get model value via property 中的 getter 裝飾器，
    # 用屬性 (object.model)的方式來讀 model value
    @property 
    def model(self):
        return self.__model.upper()
    
    @property
    def mileage(self):
        return self.__mileage
    
    @mileage.setter # setter 裝飾器，讓改 mileage 的 method 成為一個屬性 and do some check
    def mileage(self, mileage):
        if mileage < self.__mileage:
            raise ValueError('里程數不可以往下調喔!!')
        self.__mileage = mileage
    
my_car = Car('Toyota', 'Red', 1000)
print(my_car.get_model()) # ALT1 - 呼叫 get_model method
print(my_car.model)       # 呼叫 getter,  也就是 @property def model(self)
my_car.mileage = 1100     # 呼叫 setter,  也就是 @mileage.setter def mileage(self, mileage)
print(my_car.mileage)     # 呼叫 getter,  也就是 @property def mileage(self)
print(my_car)

my_car.mileage = 500     # 呼叫 setter,  也就是 @mileage.setter def mileage(self, mileage), 條件不符所以報錯

# (Supplement) Magic methods

![Special method names for operators](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*FQ4YIuvGLmMqTiuQHzpmpw.png)

In [ ]:
class Vector:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'Vector({self.x}, {self.y})'
    
    def __abs__(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
    
v1 = Vector(1, 4)
v2 = Vector(2, 0)  
v3 = v1 + v2     # call __add__ method
print(v3)
print(abs(v3))  # call __abs__ method
print(v1 * 2)   # call __mul__ method